In [ ]:
!pip install emoji

In [ ]:
import pandas as pd
import re

df = pd.read_csv('cleaned_tweets_with_emojis.csv')

# Column fix
if 'Tweet' in df.columns:
    df = df.rename(columns={'Tweet': 'text'})
if 'label' in df.columns and 'sentiment' not in df.columns:
    df = df.rename(columns={'label': 'sentiment'})
if 'text' not in df.columns:
    print("ERROR: No 'text' column found!")

print(df.head())

  Sentiment                                               text
0  Negative               Enna da ellam avan seyal  Mari iruku
1  Negative          This movei is just like  ellam avan seyal
2  Positive  Padam vanthathum 13k dislike pottavaga yellam ...
3  Positive    Neraya neraya neraya... ... V era level...thala
4  Positive  wow thavala sema mass....padam oru pundaikum a...


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string

emoji_sentiment = {"😊":"HAPPY", "😍":"LOVE", "😢":"SAD", "😡":"ANGRY", "😂":"JOY", "😭":"SAD", "😔":"SAD", "❤️":"LOVE"}
stopword_list = set(stopwords.words('english'))

def clean_text(Tweet):
    # Keep words and specified emojis (edit allowed_emojis for your full emoji set)
    allowed_emojis = "😊😍😢😡😂😭😔❤️"
    Tweet = str(Tweet).lower()
    Tweet = re.sub(r"http\S+|www\S+", "", Tweet)
    Tweet = re.sub(r"@\w+|#\w+", "", Tweet)
    # Only remove non-alphabetic (except emojis you care about)
    Tweet = re.sub(r"[^a-z\s" + allowed_emojis + "]", "", Tweet)
    words = Tweet.split()
    words = [word for word in words if word not in stopword_list and len(word) > 2]
    return " ".join(words)
# Re-create df['clean_text']
df['clean_text'] = df['text'].apply(clean_text)


# Apply cleaning
df['clean_text'] = df['text'].apply(clean_text)
print(df.head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  Sentiment                                               text  \
0  Negative               Enna da ellam avan seyal  Mari iruku   
1  Negative          This movei is just like  ellam avan seyal   
2  Positive  Padam vanthathum 13k dislike pottavaga yellam ...   
3  Positive    Neraya neraya neraya... ... V era level...thala   
4  Positive  wow thavala sema mass....padam oru pundaikum a...   

                                          clean_text  
0                   enna ellam avan seyal mari iruku  
1                        movei like ellam avan seyal  
2  padam vanthathum dislike pottavaga yellam yea ...  
3                neraya neraya neraya era levelthala  
4   wow thavala sema masspadam oru pundaikum aagathu  


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df["label"] = le.fit_transform(df["Sentiment"])
print(le.classes_)

['Mixed_feelings' 'Negative' 'Positive' 'not-Tamil' 'unknown_state']


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df["label"] = le.fit_transform(df["Sentiment"])
print(le.classes_)

from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
def emoji_word_tokenizer(text):
    return re.findall(r'\w+|[😊😍😢😡😂😭😔❤️]', text)

vectorizer = TfidfVectorizer(
    tokenizer=emoji_word_tokenizer,
    max_features=8000,
    ngram_range=(1,2)
)
X = vectorizer.fit_transform(df["clean_text"])
y = df["label"].values

# SMOTE and rest as before

smote = SMOTE(random_state=42, k_neighbors=3)
X_res, y_res = smote.fit_resample(X, y)
print("Balanced class distribution:", pd.Series(y_res).value_counts())

['Mixed_feelings' 'Negative' 'Positive' 'not-Tamil' 'unknown_state']


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Balanced class distribution: 1    9992
2    9992
0    9992
3    9992
4    9992
Name: count, dtype: int64


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(
    X_res, y_res, test_size=0.2, random_state=42, stratify=y_res
)
# Try much higher C, remove regularization penalty
logreg = LogisticRegression(
    solver='saga',
    multi_class='multinomial',
    max_iter=10000,
    C=50.0,
    penalty=None,
    class_weight={0:0.8, 1:1.0, 2:1.2, 3:1.0, 4:1.0},  # Example: 0=Mixed_feelings, 1=Negative, 2=Positive, etc.
    n_jobs=-1
)

logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f'Logistic Regression Accuracy: {acc * 100:.2f}%')
print(f'Weighted F1 Score: {f1:.4f}')
print(classification_report(y_test, y_pred, digits=4, target_names=list(le.classes_)))

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1207: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Logistic Regression Accuracy: 90.23%
Weighted F1 Score: 0.8967
                precision    recall  f1-score   support

Mixed_feelings     0.8520    0.9505    0.8986      1999
      Negative     0.8717    0.9520    0.9100      1998
      Positive     0.9516    0.6296    0.7578      1998
     not-Tamil     0.9331    0.9985    0.9647      1999
 unknown_state     0.9250    0.9810    0.9521      1998

      accuracy                         0.9023      9992
     macro avg     0.9067    0.9023    0.8967      9992
  weighted avg     0.9067    0.9023    0.8967      9992



In [ ]:
def hardcode_correction(text, pred_sentiment, pred_emotion, probas=None):
    # Emoji-based correction
    if any(e in text for e in ["😊","😍","😂"]):
        if pred_sentiment == "Mixed_feelings":
            return "Positive", pred_emotion
    if any(e in text for e in ["😡","😢"]):
        if pred_sentiment == "Mixed_feelings":
            return "Negative", pred_emotion

    # Text-only (no emojis), optionally use probabilities if available
    if probas is not None and pred_sentiment == "Mixed_feelings":
        max_proba = max(probas)
        max_idx = probas.index(max_proba)
        idx2label = {0:"Mixed_feelings", 1:"Negative", 2:"Positive"}  # adjust as per your label order
        # If model is confident about Positive/Negative, override Mixed_feelings
        if max_proba > 0.60 and max_idx in [1,2]:
            return idx2label[max_idx], pred_emotion
    return pred_sentiment, pred_emotion

test_texts = [
    "Nalla padam da sema feel varuthu 😊",
    "Intha trailer mokka da 😢",
    "Super movie da love it 😍",
    "Ippadiye poguthu mass 😂",
    "Worst padam thalaiva 😡",
    "Konjam Irumugan Madhiri irukku....All the best.....team.."
]
for txt in test_texts:
    predict_sentiment_and_emotion(txt)

Text: Nalla padam da sema feel varuthu 😊
Predicted Sentiment: Mixed_feelings
Predicted Emotion: HAPPY
--------------------------------------------------
Text: Intha trailer mokka da 😢
Predicted Sentiment: Negative
Predicted Emotion: SAD
--------------------------------------------------
Text: Super movie da love it 😍
Predicted Sentiment: Positive
Predicted Emotion: LOVE
--------------------------------------------------
Text: Ippadiye poguthu mass 😂
Predicted Sentiment: Mixed_feelings
Predicted Emotion: JOY
--------------------------------------------------
Text: Worst padam thalaiva 😡
Predicted Sentiment: Mixed_feelings
Predicted Emotion: ANGRY
--------------------------------------------------
Text: Konjam Irumugan Madhiri irukku....All the best.....team..
Predicted Sentiment: Mixed_feelings
Predicted Emotion: Neutral
--------------------------------------------------
